In [1]:
import os
import sys
import mlflow

sys.path.append(os.path.abspath(".."))

from recruitair.job_offers.models import KeyCriteriaResponse

mlflow.set_tracking_uri("http://nattech.fib.upc.edu:40380/")

In [2]:
mlflow.langchain.autolog()
mlflow.set_experiment("criteria-extraction")

<Experiment: artifact_location='mlflow-artifacts:/900495739893713381', creation_time=1759680270113, experiment_id='900495739893713381', last_update_time=1759680270113, lifecycle_stage='active', name='criteria-extraction', tags={'domain': 'recruitair',
 'mlflow.experimentKind': 'custom_model_development',
 'task': 'criteria-extraction'}>

In [3]:
MLFLOW_PROMPT_NAME = "criteria-extraction"
MLFLOW_PROMPT_VERSION = 2
OLLAMA_MODEL = "dolphin3"
OLLAMA_MODEL_VERSION = "8b"

In [4]:
import pandas as pd
import json

with open("../data/interim/preprocessed_jobs.jsonl", "r") as f:
    lines = f.readlines()

job_offers = []
criteria = []
for line in lines:
    job_offer = json.loads(line)
    job_offers.append({"job_offer_text": job_offer["job_description"]})
    criteria.append({"key_criteria": job_offer["criteria"]})

data = pd.DataFrame({"inputs": job_offers, "expectations": criteria})

In [5]:
from langchain_ollama import ChatOllama


@mlflow.trace
def predict(job_offer_text: str) -> KeyCriteriaResponse:
    llm = ChatOllama(model=f"{OLLAMA_MODEL}:{OLLAMA_MODEL_VERSION}", temperature=0)
    prompt = mlflow.genai.load_prompt(f"prompts:/{MLFLOW_PROMPT_NAME}/{MLFLOW_PROMPT_VERSION}")
    response = llm.with_structured_output(prompt.response_format, method="json_schema").invoke(
        prompt.format(job_offer_text=job_offer_text)
    )
    return KeyCriteriaResponse.model_validate(response)

In [6]:
from mlflow.entities import Feedback
import ollama

import numpy as np


@mlflow.genai.scorer(name="target-recall/embedding/mxbai-embed-large:335m")
def target_recall(outputs: KeyCriteriaResponse, expectations: KeyCriteriaResponse) -> Feedback:
    expectations = KeyCriteriaResponse.model_validate(expectations)
    # Compute the embeddings of the names of the all the extracted and target criteria:
    target_embeddings = []
    for target_criterion in expectations.key_criteria:
        target_embeddings.append(ollama.embed("mxbai-embed-large:335m", input=target_criterion.name)["embeddings"][0])
    response_embeddings = []
    for response_criterion in outputs.key_criteria:
        response_embeddings.append(
            ollama.embed("mxbai-embed-large:335m", input=response_criterion.name)["embeddings"][0]
        )
    # Compute the cosine similarity matrix between the two sets of embeddings:
    similarity_matrix = np.inner(np.array(response_embeddings), np.array(target_embeddings))

    # We'll score as follows: For each target criterion, we'll find the most similar
    # response criterion, thus we'll have, for each target criterion, a score
    # between 0 and 1 representing how well it was matched. We'll then floor
    # everything below 0.8 to 0, and average the rest.
    # This means that if a target criterion was not matched with at least 0.8,
    # it will contribute 0 to the average. We'll call this "target recall score".
    # It can be interpreted as the "rich fraction" of target criteria that were well matched.
    scores = similarity_matrix.max(axis=0)
    target_recall_score = float(np.where(scores < 0.8, 0, scores).mean())
    return Feedback(value=target_recall_score)

In [7]:
with mlflow.start_run(run_name="prompt-evaluation"):
    mlflow.log_param("ollama_model", OLLAMA_MODEL)
    mlflow.log_param("ollama_model_version", OLLAMA_MODEL_VERSION)
    mlflow.log_param("temperature", 0)
    mlflow.log_param("mlflow_prompt_name", MLFLOW_PROMPT_NAME)
    mlflow.log_param("mlflow_prompt_version", MLFLOW_PROMPT_VERSION)

    mlflow.genai.evaluate(
        predict_fn=predict,
        data=data,
        scorers=[target_recall],
    )

2025/10/05 19:39:42 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/10/05 19:39:42 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
c:\Users\mirxm\Storage\Work\MDS\S3\MLOps\CriteriaExtractor\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Evaluating:   0%|          | 0/849 [Elapsed: 00:00, Remaining: ?] 2025/10/05 19:39:51 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/10/05 19:39:51 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTER